In [131]:
import re
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

from joblib import Memory
from scipy.sparse import hstack
from scipy.sparse import save_npz,load_npz



In [132]:
# Set up memory caching
memory = Memory('./cachedir', verbose=0)

## Given:
### [Real or Fake] : Fake Job Description Prediction
### This dataset contains 18K job descriptions out of which about 800 are fake. The data consists of both textual information and meta-information about the jobs. The dataset can be used to create classification models which can learn the job descriptions which are fraudulent.

## Goal:

Process the dataset to prepare features for a classification model that predicts whether job descriptions are fraudulent or real.

## Load and explore the data:

In [133]:
@memory.cache
def load_data():
    return pd.read_csv("../data/raw/fake_job_postings.csv", index_col=0)

# Load the DataFrame (will use cache if available)
df = load_data()

In [134]:
# Display the first two rows
df.head(2)

,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
job_id,,,,,,,,,,,,,,,,,
1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0


In [135]:
df[df['fraudulent']==1].head(2)

,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
job_id,,,,,,,,,,,,,,,,,
99,IC&E Technician,"US, , Stocton, CA",Oil & Energy,95000-115000,...,"IC&amp;E Technician | Bakersfield, CA Mt. Poso...","QualificationsKnowledge, Skills &amp; Abilitie...",BENEFITSWhat is offered:Competitive compensati...,0,1,1,Full-time,Mid-Senior level,High School or equivalent,Oil & Energy,Other,1
145,Forward Cap.,NaN,NaN,NaN,NaN,The group has raised a fund for the purchase o...,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,1


In [136]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17880 entries, 1 to 17880
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   title                17880 non-null  object
 1   location             17534 non-null  object
 2   department           6333 non-null   object
 3   salary_range         2868 non-null   object
 4   company_profile      14572 non-null  object
 5   description          17879 non-null  object
 6   requirements         15184 non-null  object
 7   benefits             10668 non-null  object
 8   telecommuting        17880 non-null  int64 
 9   has_company_logo     17880 non-null  int64 
 10  has_questions        17880 non-null  int64 
 11  employment_type      14409 non-null  object
 12  required_experience  10830 non-null  object
 13  required_education   9775 non-null   object
 14  industry             12977 non-null  object
 15  function             11425 non-null  object
 16  fraudulen

In [137]:
# Evaluate the class distribution of 'fraudulent' labels in percentage
df['fraudulent'].value_counts(normalize=True)*100

fraudulent
0    95.1566
1     4.8434
Name: proportion, dtype: float64

### Identify Data Types

Identifying columns as binary, categorical or text is essential to be made in the beginning, because:
- Categorical variables often need to be encoded into numerical formats (e.g., one-hot encoding, label encoding) for machine learning algorithms, while text data typically requires different processing techniques, such as tokenization or vectorization.
- Different strategies are often employed for imputing missing values depending on whether a column is categorical or textual.

In [138]:
df.dtypes.sort_values()

telecommuting           int64
has_questions           int64
has_company_logo        int64
fraudulent              int64
company_profile        object
description            object
requirements           object
benefits               object
function               object
department             object
location               object
employment_type        object
required_experience    object
required_education     object
industry               object
salary_range           object
title                  object
dtype: object

In [139]:
# check unique values - helps to decide how to treat data: as binary, categorical or text
df.nunique().sort_values()

has_company_logo           2
has_questions              2
telecommuting              2
fraudulent                 2
employment_type            5
required_experience        7
required_education        13
function                  37
industry                 131
salary_range             874
department              1337
company_profile         1709
location                3105
benefits                6204
title                  11231
requirements           11967
description            14801
dtype: int64

In [140]:
# check values in a column
# Get value counts and truncate strings indexes
truncated_index_counts = df['requirements'].value_counts().rename(
    index=lambda x: x[:100]
)
truncated_index_counts

requirements
University degree required. TEFL / TESOL / CELTA or teaching experience preferred but not necessaryC    410
University degree required. TEFL / TESOL / CELTA or teaching experience preferred but not necessaryP    163
16-18 year olds only due to government funding.Full time availability.                                  117
Minimum Requirements:Minimum of 6 months customer service related experience requiredHigh school dip     80
University degree required. TEFL / TESOL / CELTA, and/or teaching experience preferredCanada/US pass     42
                                                                                                       ... 
About You2 or more years managing analytics infrastructure in a startup environment.Highly proficien      1
Understands what a startup is and is willing to work in such environmentUnderstanding of the sales p      1
RequirementsMarket knowledge about the UK real estate sector having lived and/or studied in the UK b      1
Job requirement

In [141]:
binary_columns = [
    'has_company_logo',
    'has_questions',
    'telecommuting',
    'fraudulent',
]

ordinal_columns = [
    'employment_type',
    'required_experience',
    'required_education',
    'salary_range'
]

nominal_columns = [
    'function',
    'industry',
    'department',
    'location',
    'title'
]

text_columns = [
    'company_profile',
    'benefits',
    'requirements',
    'description'
]

## Handle Missing Values

- drop high missing value columns
- drop rows depending on the importance of the columns and how much data is missing.
- fill missning values with placeholder or mode

Notes:

- As the dataset is very imbalanced (95.16% for 0 and 4.84% for 1), we will apply rows removal only for the majority class (0).

- Conduct data imputation with mean, mode and etc. **after** train/test split in order to prevent data leakage, ensuring that information from the test set does not influence the training process.

### Drop rows/cols

In [142]:
# Check for NaN values (in percentage) grouped by 'fraudulent' and sort the counts
df.groupby('fraudulent').apply(
    lambda group: group.isna().mean()*100, include_groups=False
).T.sort_values(by=0, ascending=False)

fraudulent,0,1
salary_range,84.453979,74.249423
department,64.746679,61.316397
required_education,44.986482,52.078522
benefits,40.249207,42.032333
required_experience,38.879746,50.230947
function,35.958622,38.914550
industry,27.201128,31.755196
employment_type,18.984366,27.829099
company_profile,15.992712,67.782910
requirements,14.940637,17.782910


In [143]:
# Drop columns with a high percentage of missing values in both classes
df.drop(columns=['salary_range', 'department'], inplace=True)

In [144]:
# drop rows with missing 'description'
df.dropna(subset=['description'], inplace=True)

### Impute missing values

In [145]:
missing_values_columns = df.columns[df.isna().sum() != 0].tolist()
missing_values_columns

['location',
 'company_profile',
 'requirements',
 'benefits',
 'employment_type',
 'required_experience',
 'required_education',
 'industry',
 'function']

In [146]:
# Impute categorical and text columns with placeholders
placeholder_map = {
    'location': 'Other',
    'company_profile': 'Missing',
    'requirements': 'Missing',
    'benefits':'See job description', # mode
    'employment_type':'Other',
    'required_experience': 'Not Applicable',
    'required_education':'Unspecified',
    'industry':'Other',
    'function': 'Other'
}

df.fillna(value=placeholder_map, inplace=True)

In [147]:
df.isna().sum().sum()

np.int64(0)

In [148]:
df.shape

(17879, 15)

## Encode Categorical Variables:

Apply appropriate encoding techniques:
- **Binary columns**: Use direct mapping (0/1).
- **Ordinal columns**: Use label encoding.
- **Nominal columns**: Use One-Hot Encoding, target encoding or frequency encoding 


In [149]:
# Define lists for different types of columns
binary_columns = ['has_company_logo', 'has_questions', 'telecommuting', 'fraudulent']
ordinal_columns = ['employment_type', 'required_experience', 'required_education']
nominal_columns = ['function', 'industry', 'location', 'benefits']
text_columns = ['title', 'requirements', 'description', 'company_profile']

In [150]:
df[binary_columns].dtypes

has_company_logo    int64
has_questions       int64
telecommuting       int64
fraudulent          int64
dtype: object

### Encode Ordinal columns using Label Encoding

In [151]:
ordinal_encoder = LabelEncoder()
for column in ordinal_columns:
    df[column] = ordinal_encoder.fit_transform(df[column])


### Encode Nominal columns

In [152]:
df[nominal_columns].nunique()

function      37
industry     132
location    3106
benefits    6204
dtype: int64

In [153]:
def frequency_encode(df, nominal_columns):
    for column in nominal_columns:
        # Calculate frequency of each category
        frequency = df[column].value_counts() / len(df)
        # Map frequencies to the column
        df[column] = df[column].map(frequency)
    return df

df = frequency_encode(df, nominal_columns)

## Select features

In [154]:
# use all columns, except fraudulent
X = df.drop(columns=['fraudulent'])
y = df['fraudulent']

## Text Data Cleaning
   - Remove special characters and unnecessary whitespace from text columns.
   - Convert text to lowercase for uniformity.
   - Remove leading/trainling whitespaces
   - Optionally, perform lemmatization or stemming on text data.

In [155]:
def clean_text(text):
    if isinstance(text, str):
        text = re.sub(r'[^a-zA-Z0-9_\s]', '', text)
        text = text.lower()
        text = text.strip()
    return text

In [156]:
text_columns

['title', 'requirements', 'description', 'company_profile']

In [157]:
# Apply the clean_text function to each specified text column
for column in text_columns:
    X[column] = X[column].apply(clean_text)

## Split the Data

In [158]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [159]:
X.info()
text_columns

<class 'pandas.core.frame.DataFrame'>
Index: 17879 entries, 1 to 17880
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   title                17879 non-null  object 
 1   location             17879 non-null  float64
 2   company_profile      17879 non-null  object 
 3   description          17879 non-null  object 
 4   requirements         17879 non-null  object 
 5   benefits             17879 non-null  float64
 6   telecommuting        17879 non-null  int64  
 7   has_company_logo     17879 non-null  int64  
 8   has_questions        17879 non-null  int64  
 9   employment_type      17879 non-null  int64  
 10  required_experience  17879 non-null  int64  
 11  required_education   17879 non-null  int64  
 12  industry             17879 non-null  float64
 13  function             17879 non-null  float64
dtypes: float64(4), int64(6), object(4)
memory usage: 2.0+ MB


['title', 'requirements', 'description', 'company_profile']

## Converting text data into TF-IDF features

In [160]:
# Define the TF-IDF vectorization function
@memory.cache
def fit_and_transform_tfidf(X_train_raw, X_test_raw, features):
    # Initialize vectorizers for each text feature
    vectorizers = {
        feature: TfidfVectorizer(stop_words='english', min_df=5, max_df=0.8, ngram_range=(1, 2))
        for feature in features
    }

    # List to store TF-IDF matrices
    tfidf_matrices_train = []
    tfidf_matrices_test = []

    # Loop through features to fit and transform
    for feature in features:
        # Fit on training data and transform both train and test data
        vectorizers[feature].fit(X_train_raw[feature])
        tfidf_matrix_train = vectorizers[feature].transform(X_train_raw[feature])
        tfidf_matrix_test = vectorizers[feature].transform(X_test_raw[feature])

        tfidf_matrices_train.append(tfidf_matrix_train)
        tfidf_matrices_test.append(tfidf_matrix_test)

    # Combine all TF-IDF matrices into one
    X_train = hstack(tfidf_matrices_train)
    X_test = hstack(tfidf_matrices_test)

    return X_train, X_test

# List of features to vectorize
features = ['description', 'requirements', 'benefits']

# Fit and transform the TF-IDF vectors (this will be cached)
X_train, X_test = fit_and_transform_tfidf(X_train, X_test, text_columns)


In [161]:
print(X_train.shape)
print(X_test.shape)

(14303, 137613)
(3576, 137613)


In [162]:
# check for imbalaced dataset - calculate percentages:
y_train_counts = y_train.value_counts()
percentages = (y_train_counts/y_train_counts.sum())*100
percentages

fraudulent
0    95.161854
1     4.838146
Name: count, dtype: float64

## Oversampling because the dataset is imbalanced

In [163]:
# Creating an instance of SMOTE
smote = SMOTE()

# Balancing the data
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

print(X_resampled.shape)
print(y_resampled.shape)

(27222, 137613)
(27222,)


In [164]:
# check for imbalaced dataset - calculate percentages:
y_resampled_counts = y_resampled.value_counts()
percentages = (y_resampled_counts/y_resampled_counts.sum())*100
percentages

fraudulent
0    50.0
1    50.0
Name: count, dtype: float64

## Save processed data

In [165]:
# Save the sparse matrix
save_npz('../data/processed/X_resampled.npz', X_resampled)

# Save y_resampled as a DataFrame (you can also use Series)
pd.Series(y_resampled).to_csv('../data/processedy_resampled.csv', index=False)

##  Load processed data

In [166]:
# Load the sparse matrix
X_resampled_loaded = load_npz('../data/processed/X_resampled.npz')

# Load the target variable
y_resampled_loaded = pd.read_csv('../data/processedy_resampled.csv').values.flatten()  # Use .values.flatten() if you need it as a 1D array